# "Hola Mundo" mediante Aprendizaje Profundo (Deep Learning)


Datos trabajados: [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist)


---

<img src="https://github.com/zalandoresearch/fashion-mnist/raw/master/doc/img/fashion-mnist-sprite.png" width="100%">


---
1. Se requiere clasificar cada una de las prendas proporcionadas por el conjunto de datos.
2. Se hace uso de Google Collective para el procesamiento de datos.
3. El modelo fue desarrollador con Keras: Red Neuronal de alto nivel


### Descomprimir los archivos de datos

In [0]:
! pip install pydrive
# these classes allow you to request the Google drive API
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file_id_test = '1NEUvBmMTn-U3Ki7KCW0sMMX3jNPwrpbm'
file_id_train = '1FzZo48e4PlcymxyQB9GHJcrLUQsobR3a'
downloaded2 = drive.CreateFile({'id': file_id_test})
downloaded1 = drive.CreateFile({'id': file_id_train})
# allows you to temporarily load your file in the notebook VM

# assume the file is called file.csv and it's located at the root of your drive
downloaded2.GetContentFile('fashion-mnist_test.csv')
downloaded1.GetContentFile('fashion-mnist_train.csv')

### Leer los datos

In [0]:
import numpy as np 
import pandas as pd

In [0]:
archivo_datos_entrenamiento = "fashion-mnist_train.csv"
archivo_datos_prueba = "fashion-mnist_test.csv"

df_entrenamiento = pd.read_csv(archivo_datos_entrenamiento)
df_prueba = pd.read_csv(archivo_datos_prueba)


In [4]:
df_entrenamiento.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_prueba.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Seleccionar todas las columnas menos la primera
atributos_entrenamiento = df_entrenamiento[df_entrenamiento.columns[1:]]
atributos_entrenamiento.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,5,0,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,1,2,0,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
atributos_prueba = df_prueba[df_prueba.columns[1:]]
atributos_prueba.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,9,8,0,...,103,87,56,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,14,53,99,17,...,0,0,0,0,63,53,31,0,0,0
3,0,0,0,0,0,0,0,0,0,161,...,137,126,140,0,133,224,222,56,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:

etiquetas_entrenamiento = df_entrenamiento['label']
etiquetas_entrenamiento.describe(include='all')

count    60000.000000
mean         4.500000
std          2.872305
min          0.000000
25%          2.000000
50%          4.500000
75%          7.000000
max          9.000000
Name: label, dtype: float64

In [9]:
etiquetas_prueba = df_prueba['label']
etiquetas_prueba.describe(include='all')

count    10000.000000
mean         4.500000
std          2.872425
min          0.000000
25%          2.000000
50%          4.500000
75%          7.000000
max          9.000000
Name: label, dtype: float64

In [10]:
atributos_entrenamiento.values.shape

(60000, 784)

In [11]:
atributos_prueba.values.shape

(10000, 784)

In [0]:
tabla_de_etiquetas = [
    'Camiseta',
    'Pantalón',
    'Pull-over',
    'Vestido',
    'Chaqueta',
    'Sandalia',
    'Camisa',
    'Deportivo',
    'Bolso',
    'Botín'
]

## Desarrollar el Clasificador usando Redes Neuronales Profundas

In [13]:
##  importando las clases y funciones que necesitaremos.
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import AveragePooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

Using TensorFlow backend.


## inicializar el generador de números aleatorios


In [0]:
seed = 7
numpy.random.seed(seed)

 ### Cargarmos los conjuntos


In [0]:
X_entrenamiento = atributos_entrenamiento.values
y_entrenamiento = etiquetas_entrenamiento.values
X_prueba        = atributos_prueba.values
y_prueba        = etiquetas_prueba.values

X_entrenamiento = X_entrenamiento.reshape(X_entrenamiento.shape[0], 1, 28, 28).astype('float32')
X_prueba = X_prueba.reshape(X_prueba.shape[0], 1, 28, 28).astype('float32')

### Normalizamos y se categorizan las salidas como One Shot




In [0]:
# normalizar las entradas de 0-255 a 0-1
X_entrenamiento = X_entrenamiento / 255
X_prueba = X_prueba / 255
# codificar salidas como "one hot"
y_entrenamiento = np_utils.to_categorical(y_entrenamiento)
y_prueba = np_utils.to_categorical(y_prueba)
num_clases = y_prueba.shape[1]

### Definimos el modelo convolucional

In [0]:
# Modelo Convolucional 
def modelo_base():
    # crear modelo
    modelo = Sequential()
    modelo.add(Conv2D(32, (5, 5), input_shape=(1, 28, 28), activation='relu')) # Creo la convolución
    modelo.add(MaxPooling2D(pool_size=(2, 2))) # hago la reducción
    modelo.add(Dropout(0.2)) # Apago las neuronas
    modelo.add(Flatten()) # Aplano
    modelo.add(Dense(128, activation='relu')) # Totalmente conectada 
    modelo.add(Dense(num_clases, activation='softmax')) # Softmax
    # Compilar modelo
    modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return modelo


## Construimos e imprimimos el modelo convolucional



In [18]:
# construir el modelo
modelo = modelo_base()
# Entrenar el modelo
modelo.fit(X_entrenamiento, y_entrenamiento, validation_data=(X_prueba, y_prueba), epochs=8, batch_size=200, verbose=2)
# Evaluación final del modelo
puntuacion = modelo.evaluate(X_prueba, y_prueba, verbose=0)
print("Error RNC: %.2f%%" % (100-puntuacion[1]*100))

Train on 60000 samples, validate on 10000 samples
Epoch 1/8
 - 6s - loss: 0.5192 - acc: 0.8162 - val_loss: 0.3788 - val_acc: 0.8703
Epoch 2/8
 - 5s - loss: 0.3429 - acc: 0.8789 - val_loss: 0.3085 - val_acc: 0.8913
Epoch 3/8
 - 5s - loss: 0.3020 - acc: 0.8912 - val_loss: 0.2853 - val_acc: 0.8981
Epoch 4/8
 - 5s - loss: 0.2759 - acc: 0.9000 - val_loss: 0.2625 - val_acc: 0.9070
Epoch 5/8
 - 5s - loss: 0.2555 - acc: 0.9064 - val_loss: 0.2521 - val_acc: 0.9093
Epoch 6/8
 - 5s - loss: 0.2395 - acc: 0.9131 - val_loss: 0.2496 - val_acc: 0.9082
Epoch 7/8
 - 5s - loss: 0.2289 - acc: 0.9151 - val_loss: 0.2391 - val_acc: 0.9140
Epoch 8/8
 - 5s - loss: 0.2128 - acc: 0.9213 - val_loss: 0.2319 - val_acc: 0.9160
Error RNC: 8.40%
